In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ccxt
from btc_model.setting.setting import get_settings
from btc_model.core.util.crypto_util import CryptoUtil


/Users/Jason/work/source/03_ThorpAI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
# 获取设置
setting = get_settings('cex.okx')

apikey = setting['apikey']
secretkey = setting['secretkey']
passphrase = setting['passphrase']
proxy = setting['proxy']

# 初始化币安交易所
params = {
    'enableRateLimit': True,
    'proxies': {
        'http': get_settings('common')['proxies']['http'],                  
        'https': get_settings('common')['proxies']['https'],
    },
    'apiKey': apikey,          
    'secret': secretkey,  
    'password': passphrase,     
    'options': {
        'defaultType': 'spot',  # 可选：'spot', 'margin', 'future'
    },
    #  'headers': {
    #     'x-simulated-trading': '1'
    # }
}

exchange = ccxt.okx(params)

# exchange.set_sandbox_mode(True)

In [59]:
try:
    # 获取账户余额
    balance = exchange.fetch_balance({
        'type': 'trading'
    })
    
    print("=== 模拟账户余额 ===")
    for currency in balance['total']:
        if balance['total'][currency] > 0:
            print(f"{currency}:")
            print(f"  可用: {balance['free'][currency]}")
            print(f"  冻结: {balance['used'][currency]}")
            print(f"  总额: {balance['total'][currency]}")

except Exception as e:
    print(f"错误: {str(e)}")


=== 模拟账户余额 ===
USDT:
  可用: 70.5004472
  冻结: 0.0
  总额: 70.5004472
LSK:
  可用: 1.99
  冻结: 0.0
  总额: 1.99


In [13]:

try:
    # 现货限价单
    def place_spot_limit_order(symbol, side, amount, price):
        """
        symbol: 交易对，如 'BTC/USDT'
        side: 'buy' 或 'sell'
        amount: 数量
        price: 价格
        """
        order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side=side,
            amount=amount,
            price=price
        )
        return order

    # 现货市价单
    def place_spot_market_order(symbol, side, amount):
        order = exchange.create_order(
            symbol=symbol,
            type='market',
            side=side,
            amount=amount
        )
        return order

    # 永续合约限价单
    def place_swap_limit_order(symbol, side, amount, price, leverage=1):
        """
        symbol: 交易对，如 'BTC/USDT:USDT'
        side: 'buy' 或 'sell'
        amount: 合约数量
        price: 价格
        leverage: 杠杆倍数
        """
        # 设置杠杆
        exchange.set_leverage(leverage, symbol)
        
        order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side=side,
            amount=amount,
            price=price,
            params={
                'tdMode': 'cross',  # 全仓模式，'isolated' 为逐仓模式
                'posSide': 'long'   # 持仓方向，'long' 或 'short'
            }
        )
        return order


    # 示例使用
    # 现货限价买入
    min_amount = 0.01  # 假设最小交易量为 0.01
    spot_order = place_spot_limit_order(
        symbol='BTC/USDT',
        side='buy',
        amount=min_amount,  # 使用最小交易量
        price=40000         # 价格 40000 USDT
    )
    print("现货限价单:", spot_order)

    # 永续合约开多
    swap_order = place_swap_limit_order(
        symbol='BTC/USDT:USDT',
        side='buy',
        amount=min_amount,  # 使用最小交易量
        price=40000,
        leverage=2          # 2倍杠杆
    )
    print("永续合约限价单:", swap_order)

except Exception as e:
    print(f"下单错误: {str(e)}")

# 查询订单状态
def check_order_status(order_id, symbol):
    try:
        order = exchange.fetch_order(order_id, symbol)
        print(f"订单状态: {order['status']}")
        return order
    except Exception as e:
        print(f"查询订单错误: {str(e)}")
        return None

# 取消订单
def cancel_order(order_id, symbol):
    try:
        result = exchange.cancel_order(order_id, symbol)
        print("订单已取消")
        return result
    except Exception as e:
        print(f"取消订单错误: {str(e)}")
        return None

现货限价单: {'info': {'clOrdId': 'e847386590ce4dBC141ae60e0dd45f79', 'ordId': '2291432507889983488', 'sCode': '0', 'sMsg': 'Order placed', 'tag': 'e847386590ce4dBC', 'ts': '1740792410329'}, 'id': '2291432507889983488', 'clientOrderId': 'e847386590ce4dBC141ae60e0dd45f79', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'triggerPrice': None, 'average': None, 'cost': None, 'amount': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'reduceOnly': False, 'fees': [], 'stopPrice': None}
下单错误: okx {"code":"1","data":[{"clOrdId":"e847386590ce4dBCa9ceb4399b2cbf5a","ordId":"","sCode":"51010","sMsg":"You can't complete this request under your current account mode. ","tag":"e847386590ce4dBC","ts":"1740792410713"}],"inTime":"1740792410713393","msg":"All operations failed",

In [14]:
# 现货市价单
def place_spot_market_order(symbol, side, amount):
    order = exchange.create_order(
        symbol=symbol,
        type='market',
        side=side,
        amount=amount
    )
    return order

# 示例使用
try:
    # 使用市价单买入
    market_order = place_spot_market_order(
        symbol='BTC/USDT',
        side='buy',
        amount=0.01  # 确保数量符合最小交易量
    )
    print("现货市价单:", market_order)

except Exception as e:
    print(f"下单错误: {str(e)}")

现货市价单: {'info': {'clOrdId': 'e847386590ce4dBCbeca8fa4f69eac40', 'ordId': '2291433857851891712', 'sCode': '0', 'sMsg': 'Order placed', 'tag': 'e847386590ce4dBC', 'ts': '1740792450561'}, 'id': '2291433857851891712', 'clientOrderId': 'e847386590ce4dBCbeca8fa4f69eac40', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'market', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'triggerPrice': None, 'average': None, 'cost': None, 'amount': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'reduceOnly': False, 'fees': [], 'stopPrice': None}


In [4]:
crypto_util = CryptoUtil.get_instance()

In [9]:
crypto_util.get_funding_rate(exchange=exchange, symbol='LSK/USDT:USDT')

{}

In [ ]:
crypto_util.get_perpetual_markets(exchange=exchange)

In [5]:
markets = exchange.load_markets()

In [6]:
perpetual_markets = {}
            
for symbol, market in markets.items():
    # 筛选永续合约
    if market.get('swap') and market.get('linear'):
        market_info = {
            'symbol': symbol,
            'base': market['base'],
            'quote': market['quote'],
            'settle': market.get('settle'),
            'leverage': {
                'max': market.get('limits', {}).get('leverage', {}).get('max'),
                'min': market.get('limits', {}).get('leverage', {}).get('min', 1)
            },
            'margin_mode': market.get('margin_modes', ['isolated', 'cross']),
            'fees': {
                'maker': market.get('maker'),
                'taker': market.get('taker'),
            },
            'maintenance_margin': market.get('maintenance_margin_rate'),
            'initial_margin': market.get('initial_margin_rate'),
            'contract_size': market.get('contractSize', 1),
            'precision': {
                'price': market['precision']['price'],
                'amount': market['precision']['amount']
            }
        }
        perpetual_markets[symbol] = market_info

In [13]:
crypto_util.get_withdrawal_fees(exchange=exchange, currencies=['BTC', 'LSK'])

{'BTC': {'fee': 0.0, 'networks': {'default': 0.0}},
 'LSK': {'fee': 0.08, 'networks': {'default': 0.08}}}

In [14]:
exchange.fetch_balance()

{'info': {'code': '0',
  'data': [{'adjEq': '',
    'borrowFroz': '',
    'details': [{'accAvgPx': '',
      'availBal': '70.5004472',
      'availEq': '',
      'borrowFroz': '',
      'cashBal': '70.5004472',
      'ccy': 'USDT',
      'clSpotInUseAmt': '',
      'crossLiab': '',
      'disEq': '70.44052181987999',
      'eq': '70.5004472',
      'eqUsd': '70.44052181987999',
      'fixedBal': '0',
      'frozenBal': '0',
      'imr': '',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '',
      'liab': '',
      'maxLoan': '',
      'maxSpotInUse': '',
      'mgnRatio': '',
      'mmr': '',
      'notionalLever': '',
      'openAvgPx': '',
      'ordFrozen': '0',
      'rewardBal': '',
      'smtSyncEq': '0',
      'spotBal': '',
      'spotCopyTradingEq': '0',
      'spotInUseAmt': '',
      'spotIsoBal': '0',
      'spotUpl': '',
      'spotUplRatio': '',
      'stgyEq': '0',
      'totalPnl': '',
      'totalPnlRatio': '',
      'twap': '0',
      '

In [17]:
exchange.fetch_positions()

[]